#### Find bug causing nearby times to predict same abundances

In [5]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
import os
FS=16
os.chdir('/home/oliverphilcox/ChempyMulti/')
from Chempy.parameter import ModelParameters


In [19]:
a.time_steps

28

In [145]:
end = a.end
old_timesteps=28
age_of_star = a.end-12.01

old_sfr = np.linspace(0,end,old_timesteps)

new_timesteps = int((a.time_steps-1)*a.end/(a.end-age_of_star)+1)
new_end = (a.end-age_of_star)*(new_timesteps-1)/(a.time_steps-1)

new_sfr = np.linspace(0,new_end,new_timesteps)

In [146]:
new_sfr

array([ 0.        ,  0.44481481,  0.88962963,  1.33444444,  1.77925926,
        2.22407407,  2.66888889,  3.1137037 ,  3.55851852,  4.00333333,
        4.44814815,  4.89296296,  5.33777778,  5.78259259,  6.22740741,
        6.67222222,  7.11703704,  7.56185185,  8.00666667,  8.45148148,
        8.8962963 ,  9.34111111,  9.78592593, 10.23074074, 10.67555556,
       11.12037037, 11.56518519, 12.01      , 12.45481481, 12.89962963,
       13.34444444, 13.78925926])

In [139]:
np.linspace(0,1,5)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [137]:
1*(3-1)/(5-1)

0.5

In [133]:
np.linspace(0.,1.,int(4/2))

array([0., 1.])

In [127]:
old_sfr

array([ 0.        ,  0.51102284,  1.02204568,  1.53306852,  2.04409136,
        2.5551142 ,  3.06613704,  3.57715988,  4.08818272,  4.59920555,
        5.11022839,  5.62125123,  6.13227407,  6.64329691,  7.15431975,
        7.66534259,  8.17636543,  8.68738827,  9.19841111,  9.70943395,
       10.22045679, 10.73147963, 11.24250247, 11.75352531, 12.26454815,
       12.77557099, 13.28659382, 13.79761666])

In [151]:
a=ModelParameters()
from Chempy.sfr import SFR
from Chempy.solar_abundance import solar_abundances
def shorten_sfr2(a,age=None):
    '''
    This function crops the SFR to the length of the age of the star and ensures that enough stars are formed at the stellar birth epoch

    INPUT:

    a = Modelparameters

    OUTPUT:
    
    the function will update the modelparameters, such that the simulation will end when the star is born and it will also check whether there is enough sfr left at that epoch
    '''
    #try:
    #    star = np.load('%s.npy' %(a.stellar_identifier))
    #except Exception as ex:
    #    from . import localpath
    #    star = np.load(localpath + 'input/stars/' + a.stellar_identifier + '.npy')
    if age==None:
        age_of_star = star['age'][0]
    else:
        age_of_star=age
    assert (age_of_star <= 13.0), "Age of the star must be below 13Gyr"
    
    new_timesteps = int((a.time_steps-1)*a.end/(a.end-age_of_star)+1)
    new_end = (a.end-age_of_star)*(new_timesteps-1)/(a.time_steps-1)
    print(new_timesteps,new_end)
    
    basic_sfr = SFR(a.start,new_end,new_timesteps)
    
    if a.basic_sfr_name == 'gamma_function':
        getattr(basic_sfr, a.basic_sfr_name)(S0 = a.S_0 * a.mass_factor,a_parameter = a.a_parameter, loc = a.sfr_beginning, scale = a.sfr_scale)
    elif a.basic_sfr_name == 'model_A':
        basic_sfr.model_A(a.mass_factor*a.S_0,a.t_0,a.t_1)
    elif a.basic_sfr_name == 'prescribed':
        basic_sfr.prescribed(a.mass_factor, a.name_of_file)
    elif a.basic_sfr_name == 'doubly_peaked':
        basic_sfr.doubly_peaked(S0 = a.mass_factor*a.S_0, peak_ratio = a.peak_ratio, decay = a.sfr_decay, t0 = a.sfr_t0, peak1t0 = a.peak1t0, peak1sigma = a.peak1sigma)
    basic_sfr.sfr = a.total_mass * np.divide(basic_sfr.sfr,sum(basic_sfr.sfr))
    mass_normalisation = a.total_mass
    mean_sfr = sum(basic_sfr.sfr) / new_end
    
    # at which time in the simulation is the star born
    star_time = a.end-age_of_star#basic_sfr.t[-1] - age_of_star
    cut = [np.where(np.abs(basic_sfr.t - star_time) == np.min(np.abs(basic_sfr.t - star_time)))]
    if len(cut[0][0]) != 1:
        cut = cut[0][0][0]
    
    # updating the end time and the model steps and rescale the total mass
    time_model = float(basic_sfr.t[cut])
    a.end = time_model
    a.time_steps = int(cut[0][0]) + 1
    a.total_mass = sum(basic_sfr.sfr[0:a.time_steps])
    
    # check whether the sfr is enough at end to produce reasonable number of stars (which is necessary in order to have a probability to observe a star at all)
    sfr_at_end = float(basic_sfr.sfr[cut] / basic_sfr.dt)
    fraction_of_mean_sfr = sfr_at_end / mean_sfr
    if fraction_of_mean_sfr<0.05:
        return np.inf
    else:
        return a
    #assert fraction_of_mean_sfr > 0.05, ('The total SFR of the last age bin is below 5% of the mean SFR', 'stellar identifier = ', a.stellar_identifier, 'star time = ', star_time, 'model time = ', time_model ) 
    

In [152]:
from Chempy.wrapper import Chempy_all_times
def cem_real2_single_time2(a,this_time):
    """ real chempy function for returning predictions at a single timestep."""
    age=a.end-this_time
    #a.end = this_time # reset a.end variable
    a = shorten_sfr2(a,age) # use 13 max possible time her
    if a==np.inf:
        return np.inf
    
    basic_solar = solar_abundances()
    getattr(basic_solar,a.solar_abundance_name)() # need for normalizations
    
    elements_to_trace = list(a.elements_to_trace)
    
    cube, abundances = Chempy_all_times(a)
    if type(cube)==float:
        # Something became negative - discard this run.
        return np.inf
    cube1 = cube.cube
    gas_reservoir = cube.gas_reservoir
        
    abundance_list = np.zeros(len(elements_to_trace))
        
    # predicted values are written out and returned together with corona metallicity and SN-ratio
    #abundance_list = []
    for i,item in enumerate(elements_to_trace):
        abundance_list[i]=abundances[item][-1]

    """
    
    abundance_list[-2]=gas_reservoir['Z'][1:]
    elements_to_trace.append('Zcorona')

    abundance_list[-1]=np.divide(cube1['sn2'][1:],cube1['sn1a'][1:])
    elements_to_trace.append('SNratio')
    """
    return(abundance_list, elements_to_trace)

In [153]:
from Chempy.cem_function import cem_real2_single_time,extract_parameters_and_priors
def single_timestep_chempy2(args):
    """
    Calls Chempy and returns posterior values for a single pre-determined timestep, given in args.
    """
    all_parameter,a=args
    changing_parameter=all_parameter[:-1] # physics parameters
    birth_time = all_parameter[-1] # time of star birth
    
    # Update the parameters + priors:
    a = extract_parameters_and_priors(changing_parameter,a)
    
    # Save the output time etc. which is changed by the actual calculation but restored to default afterwards
    backup = a.end, a.time_steps, a.total_mass
    
    # Call Chempy and return all abundances and the list of element names
    output=cem_real2_single_time2(a,birth_time)
    if output==np.inf:
        return np.inf
    else:
        abundance_list, elements_to_trace = output
    
        # Restore default values
        a.end, a.time_steps, a.total_mass = backup
    
        return abundance_list, elements_to_trace


In [154]:
runner = lambda time: single_timestep_chempy2((list(a.p0)+[time],a))[0]
print(runner(12.)-runner(12.01))

32 13.777777777777779


/home/oliverphilcox/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/oliverphilcox/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


32 13.789259259259259
[ 0.00015044 -0.00038447  0.00038447  0.00034773  0.00016193  0.00014823
  0.00015811  0.00016962  0.00013277]
